# Import bibliotek i danych

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from operator import itemgetter
import time
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn import neighbors
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor 
from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn import linear_model
import pickle
with open("../data/model_KNN.p", "rb") as fp:
    modelkNN = pickle.load(fp)
with open("../data/model_OLS.p", "rb") as fp:
    modelOLS = pickle.load(fp)
with open("../data/model_XGB.p", "rb") as fp:
    modelXGBoost = pickle.load(fp)
    
df = pd.read_csv('../data/data_eda.csv')
df=df.rename(columns={'Unnamed: 0': 'index_0'})
target = "stars"
SEED = 17

$\text{Podział danych na zbiór treningowy i testowy}$

In [2]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=SEED)

In [3]:
test_data

,index_0,pages,stars,reviews,series,mix,character,plot,funny,lighthearted,...,author_stars,Fiction,Nonfiction,Literary,Fantasy,Crime,Social,Children,Romans,Realism
4634,4634,485,4.12,10915,0,0.25,0.70,0.03,0.20,0.02,...,3.760000,1,0,0,0,0,1,1,0,1
584,584,336,4.73,18604,0,0.00,0.00,0.00,0.00,0.00,...,4.705000,0,1,0,0,0,1,0,0,1
928,928,344,3.66,33000,0,0.39,0.11,0.48,0.01,0.00,...,3.475000,1,0,0,0,1,0,0,0,0
1918,1918,466,3.77,18884,0,0.57,0.09,0.32,0.08,0.02,...,3.993333,1,0,0,1,0,0,1,0,0
4255,4255,368,3.79,2891,1,0.67,0.12,0.20,0.57,0.58,...,3.700000,1,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1990,1990,304,3.75,3739,0,0.55,0.28,0.15,0.64,0.14,...,0.000000,1,0,0,0,1,0,0,0,0
1178,1178,720,3.67,6069,0,0.53,0.17,0.28,0.04,0.00,...,3.865882,1,0,0,0,0,0,0,0,0
4155,4155,416,3.93,3097,1,0.54,0.19,0.25,0.20,0.18,...,4.060000,1,0,0,1,0,0,1,0,0
6893,6893,484,3.64,2102,1,0.56,0.21,0.21,0.04,0.00,...,3.780000,1,0,0,1,0,0,0,1,0


Tworzymy dataframe z predykcjami wszystkich modeli

In [4]:
predNames = []
for result in [modelXGBoost, modelkNN, modelOLS]:
    # Tworzymy wektor predykcji tworząc serię pandasa, a następnie sortujemy indeks
    pred = pd.Series(result["predictions"], index=result["indices"]).sort_index()
    # Dodajmy kolumnę z predykcjami na podstawie nazwy z pliku
    # Wyczyścimy nazwy modeli aby zlikwidować spacje i pozbyć się niepotrzebnych składników
    cleanName = (result['name']).replace(" ", "_").split("_")[0]
    predNames.append(f"pred_{cleanName}")
    test_data[f"pred_{cleanName}"] = pred

In [5]:
test_data[predNames].corr()

,pred_XGB,pred_KNN,pred_OLS
pred_XGB,1.000000,0.832026,0.855591
pred_KNN,0.832026,1.000000,0.761616
pred_OLS,0.855591,0.761616,1.000000


Najmniejsza korelacja między KNN i OLS, dla reszty dość wysoka.

In [6]:
test_data[predNames].sample(10)

,pred_XGB,pred_KNN,pred_OLS
1807,4.549220,4.528698,4.496634
139,3.779438,3.750481,3.925831
4135,3.943887,3.965090,3.930335
1178,3.930177,3.870477,3.950681
5635,3.662794,3.793440,3.660916
547,4.509120,4.291554,4.542659
1176,4.043925,3.889883,4.146053
6372,3.861532,3.940164,3.810099
3382,4.071838,4.027596,3.908453
986,4.273121,4.354500,4.260874


Ale nadal widać różnice w predykcjach, sprawdźmy jakość predykcji średniej z modeli:

In [7]:
test_data["pred_mean"] = test_data[['pred_XGB','pred_KNN','pred_OLS']].mean(axis=1)

In [8]:
metrics.mean_squared_error(test_data['stars'], test_data['pred_mean'])**0.5

0.19991201582530285

Jakość predykcji lepsza niż dla kNN i OLS osobno, ale gorsza niż dla XGBoost, więc spróbujmy zmienić wagi na współczynniki z regresji:

In [9]:
from statsmodels.api import OLS
OLS(test_data['stars'],test_data[['pred_XGB','pred_KNN','pred_OLS']]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  stars   R-squared (uncentered):                   0.998
Model:                            OLS   Adj. R-squared (uncentered):              0.998
Method:                 Least Squares   F-statistic:                          1.849e+05
Date:                Sat, 11 May 2024   Prob (F-statistic):                        0.00
Time:                        19:38:50   Log-Likelihood:                          291.71
No. Observations:                1386   AIC:                                     -577.4
Df Residuals:                    1383   BIC:                                     -561.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
pred_XGB       0.7106      0.052     13.750      0.000       0.609       0.812
pred_KNN       0.1136      0.043      2.659      0.008       0.030       0.197
pred_OLS       0.1754      0.045      3.864      0.000       0.086       0.264
==============================================================================
Omnibus:                       40.752   Durbin-Watson:                   1.984
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               51.818
Skew:                          -0.334   Prob(JB):                     5.60e-12
Kurtosis:                       3.672   Cond. No.                         82.0
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
test_data["pred_regg"] = test_data['pred_XGB']*	0.7106+test_data['pred_KNN']*0.1136+test_data['pred_OLS']*0.1754
metrics.mean_squared_error(test_data['stars'], test_data['pred_regg'])**0.5

0.19604609414095647

Ostateczny wynik lepszy niż dla samego XGBoost: (z 0.1983 na 0.19604)